In [11]:
import geopandas as gpd

In [12]:
import pandas as pd

crime_df = pd.read_csv('year 2019.csv')

C:\Users\dell\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
crime_gdf = gpd.GeoDataFrame(crime_df, geometry=gpd.points_from_xy(crime_df['Long'], crime_df['Lat']))

In [14]:
gpd.points_from_xy(crime_df['Long'], crime_df['Lat'])

 ...]

In [15]:
crime_gdf = gpd.GeoDataFrame(crime_df, geometry=gpd.points_from_xy(crime_df['Long'], crime_df['Lat']))
crime_gdf.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,geometry
0,102091671,2647,NaN,THREATS TO DO BODILY HARM,B3,417,0,2019/11/12 12:00,2019,11,Tuesday,12,NaN,MORA ST,42.282082,-71.073649,"(42.28208197671972, -71.07364874515646)",POINT (-71.07365 42.28208)
1,102095489,3115,NaN,INVESTIGATE PERSON,E18,520,0,2019/11/25 16:30,2019,11,Monday,16,NaN,POYDRAS ST,42.256216,-71.124019,"(42.256215920402155, -71.12401947329023)",POINT (-71.12402 42.25622)
2,102096818,2905,NaN,VAL - VIOLATION OF AUTO LAW,A1,,0,2019/11/30 21:00,2019,11,Saturday,21,NaN,SUDBURY ST & CAMBRIDGE ST\nBOSTON MA 02108\nU...,42.360866,-71.061316,"(42.360866027118476, -71.0613160019785)",POINT (-71.06132 42.36087)
3,129092894,3201,NaN,PROPERTY - LOST/ MISSING,NaN,503,0,2019/11/16 13:30,2019,11,Saturday,13,NaN,AMERICAN LEGION HWY,42.284467,-71.111831,"(42.28446742674232, -71.11183088758158)",POINT (-71.11183 42.28447)
4,129099920,3301,NaN,VERBAL DISPUTE,B2,330,0,2019/12/12 7:50,2019,12,Thursday,7,NaN,COLUMBIA ROAD,0.000000,0.000000,"(0, 0)",POINT (0.00000 0.00000)


In [16]:
zip_gdf = gpd.read_file('ZIP_Codes.geojson')
print(zip_gdf.shape)
print(zip_gdf.columns)

(43, 5)
Index(['OBJECTID', 'ZIP5', 'ShapeSTArea', 'ShapeSTLength', 'geometry'], dtype='object')


In [17]:
crime_with_zip_gdf = gpd.sjoin(crime_gdf, zip_gdf,op="within",how='left')


C:\Users\dell\Anaconda3\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:4326'})
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [29]:
crime_count_by_zip = crime_with_zip_gdf.groupby('OBJECTID').agg('INCIDENT_NUMBER').count().reset_index()
crime_count_by_zip['OBJECTID'] = crime_count_by_zip['OBJECTID'].astype(int)

crime_count_by_zip

,OBJECTID,INCIDENT_NUMBER
0,1,1947
1,2,4344
2,3,988
3,4,5330
4,5,4731
5,6,1045
6,8,540
7,9,3793
8,10,4786
9,11,7557


In [97]:
crime_count_gdf = crime_count_by_zip[['OBJECTID', 'INCIDENT_NUMBER']].merge(crime_with_zip_gdf[['OBJECTID', 'geometry']], how='inner', on='OBJECTID')
crime_count_gdf

,OBJECTID,INCIDENT_NUMBER,geometry
0,1,1947,POINT (-71.13907 42.35380)
1,1,1947,POINT (-71.13763 42.35246)
2,1,1947,POINT (-71.12938 42.35074)
3,1,1947,POINT (-71.13261 42.35301)
4,1,1947,POINT (-71.13515 42.35896)
...,...,...,...
83671,42,10,POINT (-71.14432 42.29742)
83672,42,10,POINT (-71.14432 42.29742)
83673,42,10,POINT (-71.14432 42.29742)
83674,42,10,POINT (-71.14298 42.29930)


In [45]:
import folium

In [113]:
clon, clat = crime_gdf['Long'].mean(), crime_gdf['Lat'].mean()
m = folium.Map(location=(clat, clon), zoom_start=11, width=800, height=400)
m

In [134]:
folium.Choropleth(
    geo_data=crime_count_gdf
    name="choropleth",
    data=crime_count_gdf
    columns= ["OBJECTID","INCIDENT_NUMBER"],
    key_on="feature.properties.OBJECTID",
    fill_color="YlGn", #‘BuGn’, ‘BuPu’, ‘GnBu’, ‘OrRd’, ‘PuBu’, ‘PuBuGn’, ‘PuRd’, ‘RdPu’, ‘YlGn’, ‘YlGnBu’, ‘YlOrBr’, and ‘YlOrRd’.
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Annual crime count by district, Boston, 2019").add_to(m)

folium.LayerControl().add_to(m)
display(m)

TypeError: float() argument must be a string or a number, not 'Point'